# Constructing linear model for OER adsorption energies
---


In [1]:
# 12 PCA components are ideal with an MAE of 0.1872

### Import Modules

In [2]:
import os
print(os.getcwd())
import sys
import time; ti = time.time()

import pickle

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

import plotly.graph_objects as go

# #########################################################
from methods import (
    get_df_features_targets,
    get_df_features_targets_seoin,
    )

from methods_models import ModelAgent, GP_Regression

from proj_data import adsorbates
from proj_data import layout_shared
from proj_data import scatter_marker_props

/home/raulf2012/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_OER/workflow/model_building/gaussian_process/my_data/all_features_mine/with_new_octahedra_info
RegressionModel_2 will eventually replace  RegressionModel_1


In [3]:
from methods import isnotebook    
isnotebook_i = isnotebook()
if isnotebook_i:
    from tqdm.notebook import tqdm
    verbose = True
    show_plot = True
else:
    from tqdm import tqdm
    verbose = False
    show_plot = False

In [4]:
root_dir = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/model_building/gaussian_process/my_data/all_features_mine")

### Script Inputs

In [5]:
target_ads_i = "oh"

feature_ads_i = "o"

use_seoin_data = False

if use_seoin_data:
    feature_ads_i = "o"

In [6]:
quick_easy_settings = False
if quick_easy_settings:
    k_fold_partition_size = 170
    do_every_nth_pca_comp = 8
else:
    k_fold_partition_size = 10
    do_every_nth_pca_comp = 1

In [7]:
# # TEMP
# print(111 * "TEMP | ")
# do_every_nth_pca_comp = 9

### Read Data

In [8]:
# #########################################################
df_features_targets = get_df_features_targets()

# #########################################################
df_seoin = get_df_features_targets_seoin()

### Combine mine and Seoin's data

In [9]:
if use_seoin_data:
    # Replace multiindex with index of tuples so that my data and Seoin's data can be combined
    indices = df_features_targets.index.tolist()
    df_features_targets.index = indices

    indices = df_seoin.index.tolist()
    df_seoin.index = indices

    # Remove columns that aren't shared by my and Seoin's data
    cols_0 =df_features_targets.columns.tolist()
    cols_1 = df_seoin.columns.tolist()

    cols_comb = cols_0 + cols_1

    cols_comb_unique = []
    for col_i in cols_comb:
        if col_i not in cols_comb_unique:
            cols_comb_unique.append(col_i)

    shared_cols = []
    for col_i in cols_comb_unique:
        if col_i in df_features_targets.columns and col_i in df_seoin.columns:
            shared_cols.append(col_i)

    # Combine data
    df_data = pd.concat([
        df_features_targets[shared_cols],
        df_seoin[shared_cols],
        ], axis=0)
else:
    df_data = df_features_targets

In [10]:
# # TEMP
# print(222 * "TEMP | ")

# df_data = df_data[df_data.data.stoich == "AB3"]

### Choosing feature columns

In [11]:
# df_data.columns.tolist()
# as_ir_opp_bl_ratio

In [12]:
# assert False

In [13]:
# df_data = df_data[[

#     # ('targets', 'g_o', ''),
#     ('targets', 'g_oh', ''),

#     ('data', 'stoich', ''),
#     ('data', 'job_id_o', ''),
#     ('data', 'job_id_oh', ''),
#     ('data', 'job_id_bare', ''),

#     # ('features', 'o', 'O_magmom'),
#     # ('features', 'o', 'Ir_magmom'),
#     # ('features', 'o', 'Ir*O_bader'),
#     ('features', 'o', 'Ir_bader'),
# #     ('features', 'o', 'O_bader'),

#     ('features', 'o', 'active_o_metal_dist'),
#     # ('features', 'o', 'angle_O_Ir_surf_norm'),
#     ('features', 'o', 'ir_o_mean'),
#     ('features', 'o', 'ir_o_std'),

#     ('features', 'o', 'octa_vol'),
#     ('features', 'o', 'p_band_center'),  # HELPS
#     # ('features', 'o', 'Ir*O_bader/ir_o_mean'),

#     # ('features', 'o', 'closest_Ir_dist'),
#     ('features', 'o', 'closest_O_dist'),


#     ('features', 'dH_bulk', ''),
#     ('features', 'volume_pa', ''),
#     ('features', 'bulk_oxid_state', ''),
#     ('features', 'effective_ox_state', ''),
#     # ('features', 'surf_area', ''),

#     # Octahedra features
#     ('features', 'o', 'oxy_opp_as_bl'),
#     ('features', 'o', 'degrees_off_of_straight__as_opp'),
#     # ('features', 'o', 'as_ir_opp_bl_ratio'),


#     ('features_pre_dft', 'active_o_metal_dist__pre', ''),
#     # ('features_pre_dft', 'ir_o_mean__pre', ''),
# #     ('features_pre_dft', 'ir_o_std__pre', ''),  # HERE
#     # ('features_pre_dft', 'octa_vol__pre', ''),

#     ]]

In [14]:
df_data = df_data[[

    # ('targets', 'g_o', ''),
    ('targets', 'g_oh', ''),

    ('data', 'stoich', ''),
    ('data', 'job_id_o', ''),
    ('data', 'job_id_oh', ''),
    ('data', 'job_id_bare', ''),

#     ('features', 'o', 'O_magmom'),
#     ('features', 'o', 'Ir_magmom'),
#     ('features', 'o', 'Ir*O_bader'),
#     ('features', 'o', 'Ir_bader'),
#     ('features', 'o', 'O_bader'),

    ('features', 'o', 'active_o_metal_dist'),
    # ('features', 'o', 'angle_O_Ir_surf_norm'),
    ('features', 'o', 'ir_o_mean'),
    # ('features', 'o', 'ir_o_std'),

    ('features', 'o', 'octa_vol'),
    # ('features', 'o', 'p_band_center'),  # HELPS
    # ('features', 'o', 'Ir*O_bader/ir_o_mean'),

    # ('features', 'o', 'closest_Ir_dist'),
    # ('features', 'o', 'closest_O_dist'),


    ('features', 'dH_bulk', ''),
    # ('features', 'volume_pa', ''),
    ('features', 'bulk_oxid_state', ''),
    ('features', 'effective_ox_state', ''),
    # ('features', 'surf_area', ''),

    # Octahedra features
    ('features', 'o', 'oxy_opp_as_bl'),
    ('features', 'o', 'degrees_off_of_straight__as_opp'),
    # ('features', 'o', 'as_ir_opp_bl_ratio'),


    ('features_pre_dft', 'active_o_metal_dist__pre', ''),
    ('features_pre_dft', 'ir_o_mean__pre', ''),
    ('features_pre_dft', 'ir_o_std__pre', ''),  # HERE
    ('features_pre_dft', 'octa_vol__pre', ''),

    ]]

In [15]:
# df_data.features.columns

df_data.columns.tolist()

[('targets', 'g_oh', ''),
 ('data', 'stoich', ''),
 ('data', 'job_id_o', ''),
 ('data', 'job_id_oh', ''),
 ('data', 'job_id_bare', ''),
 ('features', 'o', 'active_o_metal_dist'),
 ('features', 'o', 'ir_o_mean'),
 ('features', 'o', 'octa_vol'),
 ('features', 'dH_bulk', ''),
 ('features', 'bulk_oxid_state', ''),
 ('features', 'effective_ox_state', ''),
 ('features', 'o', 'oxy_opp_as_bl'),
 ('features', 'o', 'degrees_off_of_straight__as_opp'),
 ('features_pre_dft', 'active_o_metal_dist__pre', ''),
 ('features_pre_dft', 'ir_o_mean__pre', ''),
 ('features_pre_dft', 'ir_o_std__pre', ''),
 ('features_pre_dft', 'octa_vol__pre', '')]

In [16]:
kdict = [
    {
        "type": "gaussian",
        "dimension": "single",
        "width": 1.8,
        "scaling": 0.5,
        "scaling_bounds": ((0.0001, 10.),),
        }
    ]

GP_R = GP_Regression(
    kernel_list=kdict,
    regularization=0.01,
    optimize_hyperparameters=True,
    scale_data=False,
    )

In [17]:
# do_every_nth_pca_comp

In [18]:
# num_feat_cols

In [19]:
data_dict_list = []
num_feat_cols = df_data.features.shape[1]
# for num_pca_i in range(1, num_feat_cols + 1, do_every_nth_pca_comp):
for num_pca_i in range(5, num_feat_cols + 1, do_every_nth_pca_comp):

    if verbose:
        print("")
        print(40 * "*")
        print(num_pca_i)

    MA = ModelAgent(
        df_features_targets=df_data,
        Regression=GP_R,
        Regression_class=GP_Regression,
        use_pca=True,
        num_pca=num_pca_i,
        adsorbates=adsorbates,
        stand_targets=False,  # True was giving much worse errors, keep False
        )

    MA.run_kfold_cv_workflow(
        k_fold_partition_size=k_fold_partition_size,
        )

    if MA.can_run:
        if verbose:
            print("MAE:", np.round(MA.mae, 4))
            print("MA.r2:", np.round(MA.r2, 4))
            print("MAE (in_fold):", np.round(MA.mae_infold, 4))

    data_dict_i = dict()
    data_dict_i["num_pca"] = num_pca_i
    data_dict_i["MAE"] = MA.mae
    data_dict_i["ModelAgent"] = MA
    data_dict_list.append(data_dict_i)

df_models = pd.DataFrame(data_dict_list)
df_models = df_models.set_index("num_pca")




# #########################################################
# Finding best performing model
row_models_i = df_models.sort_values("MAE").iloc[0]

MA_best = row_models_i.ModelAgent

print(4 * "\n")
if verbose:
    print(
        row_models_i.name,
        " PCA components are ideal with an MAE of ",
        np.round(
        row_models_i.MAE,
            4),
        sep="")


****************************************
2
MAE: 0.2296
MA.r2: 0.6521
MAE (in_fold): 0.2569

****************************************
3
MAE: 0.1987
MA.r2: 0.7281
MAE (in_fold): 0.2574

****************************************
4
MAE: 0.1894
MA.r2: 0.7514
MAE (in_fold): 0.2072

****************************************
5
MAE: 0.1823
MA.r2: 0.7603
MAE (in_fold): 0.1575

****************************************
6
MAE: 0.171
MA.r2: 0.7985
MAE (in_fold): 0.137

****************************************
7
MAE: 0.1633
MA.r2: 0.8111
MAE (in_fold): 0.129

****************************************
8
MAE: 0.164
MA.r2: 0.8076
MAE (in_fold): 0.1272





7 PCA components are ideal with an MAE of 0.1633


In [20]:
# 7 PCA components are ideal with an MAE of 0.1604

In [21]:
# 7 PCA components are ideal with an MAE of 0.1646

In [22]:
assert False

AssertionError: 

In [ ]:
print(
    "Num features: ",
    len(df_data.features.columns.tolist()),
    "\n",
    sep="")

adsorbates_features = []
other_features = []
for i in df_data.features.columns.tolist():
    # print(i)
    if i[0] in adsorbates:
        adsorbates_features.append(i[1])
    else:
        other_features.append(i[0])

print(
    "Adsorbates features: ",
    ", ".join(
        sorted(adsorbates_features, key = lambda s: s.casefold())
        ),
    sep="")


print(
    "O features: ",
    ", ".join(list(np.sort(other_features))),
    sep="")

In [ ]:
# assert False

In [ ]:
# import plotly.graph_objs as go

# trace = go.Scatter(
#     mode="markers",
#     x=df_data.features.o.closest_Ir_dist,
#     y=df_data.features.o.closest_O_dist,
#     )
# data = [trace]

# fig = go.Figure(data=data)
# fig.show()

In [ ]:
# 11 PCA components are ideal with an MAE of 0.1703
# 11 PCA components are ideal with an MAE of 0.171
# 10 PCA components are ideal with an MAE of 0.1685

In [ ]:
from methods_models import ModelAgent_Plotter

MA_Plot = ModelAgent_Plotter(
    ModelAgent=MA_best,
    layout_shared=layout_shared,
    )

MA_Plot.plot_residuals()
MA_Plot.plot_parity()
MA_Plot.plot_parity_infold()

# # Uncomment to run pca analysis on in-fold regression
# MA.run_pca_analysis()

In [ ]:
fig = MA_Plot.plot_residuals__PLT
if show_plot:
    fig.show()

In [ ]:
fig = MA_Plot.plot_parity__PLT
if show_plot:
    fig.show()

In [ ]:
fig = MA_Plot.plot_parity_infold__PLT
if show_plot:
    fig.show()

In [ ]:
from methods_models import plot_mae_vs_pca
plot_mae_vs_pca(
    df_models=df_models,
    layout_shared=layout_shared,
    scatter_marker_props=scatter_marker_props,
    )

In [ ]:
assert False

### Save Data

In [ ]:
# Deleting cinv matrix of GP model to save disk space

for num_pca, row_i in df_models.iterrows():
    MA = row_i.ModelAgent
    MA.cleanup_for_pickle()

In [ ]:
data_dict_out = {
    "df_models": df_models,
    "ModelAgent_Plot": MA_Plot,
    }

In [ ]:
assert False

In [ ]:
# Pickling data ###########################################
directory = os.path.join(root_dir, "out_data")
print(directory)
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "modelling_data_NEW_88.pickle"), "wb") as fle:
    pickle.dump(data_dict_out, fle)
# #########################################################

In [ ]:
# #########################################################
print(20 * "# # ")
print("All done!")
print("Run time:", np.round((time.time() - ti) / 60, 3), "min")
print("model__mine_GP.ipynb")
print(20 * "# # ")
# #########################################################

In [ ]:
# df_data.features.o.

In [ ]:
# assert False

In [ ]:
# df_data

In [ ]:
# assert False